## Step 1: Verify GPU is Available

In [ ]:
# Check GPU availability
!nvidia-smi

**Expected output:** Should show GPU info (Tesla T4, L4, or similar)

**If you see an error:** Go back and enable GPU runtime!

## Step 2: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Action required:** Click the link and authorize access to your Google Drive

## Step 3: Clone Project from GitHub

In [ ]:
%cd /content
!git clone https://github.com/hmolhem/nthu-driver-drowsiness-ROI.git
%cd nthu-driver-drowsiness-ROI
!git status

**Expected:** Shows "On branch main" and clean working tree

## Step 4: Set Up Dataset Symlink

This links your Drive dataset to the project folder

In [ ]:
# Create datasets directory
!mkdir -p datasets

# Create symlink to your Drive dataset
# Adjust the path if your dataset is in a different location
!ln -s /content/drive/MyDrive/datasets/archive datasets/archive

# Verify the symlink works
!ls -lh datasets/
!ls datasets/archive/drowsy | head -3

**Expected:** Should list 3 image files from drowsy folder

**If error:** Adjust the path in the `ln -s` command to match your Drive structure

## Step 5: Verify Data Splits

In [ ]:
# Check splits exist
!ls -lh data/splits/
!head -3 data/splits/train.csv

**Expected:** Shows train.csv, val.csv, test.csv with sizes and first few rows

## Step 6: Install Dependencies

In [ ]:
# Install PyTorch with CUDA support
!pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision -U

# Install other requirements
!pip install -r requirements.txt

**Note:** May see some warnings about version conflicts — these are usually harmless

## Step 7: Verify CUDA PyTorch

In [ ]:
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU device:', torch.cuda.get_device_name(0))
    print('GPU memory:', torch.cuda.get_device_properties(0).total_memory / 1e9, 'GB')
else:
    print('WARNING: CUDA not available! Go enable GPU runtime.')

**Expected:** 
- `CUDA available: True`
- GPU device: Tesla T4, L4, or similar

**If False:** Stop here and enable GPU runtime!

## Step 8: Start Training (ResNet50 Baseline)

This will take ~30-60 minutes depending on GPU

In [ ]:
# Train with full baseline config (224px, 50 epochs with early stopping)
!python src/training/train_baseline.py --config configs/baseline_resnet50.yaml --device cuda

**What to expect:**
- Progress bars for each epoch
- Training loss should decrease over time
- Validation metrics printed after each epoch
- Early stopping may trigger before 50 epochs if val metric plateaus
- Checkpoints saved to `checkpoints/baseline_resnet50_best.pth` and `_last.pth`

## Step 9: Verify Checkpoints

In [ ]:
!ls -lh checkpoints/

**Expected:** Should see `baseline_resnet50_best.pth` and `baseline_resnet50_last.pth` (~90 MB each)

## Step 10: Evaluate on Test Set

In [ ]:
import torch
from src.models.classifier import create_model
from src.data.transforms import get_val_transforms
from src.data.dataset import create_dataloaders
from src.utils.config import get_config
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load config
cfg = get_config("configs/baseline_resnet50.yaml")
cfg.data.num_workers = 2  # Colab-friendly

# Create test dataloader
val_tf = get_val_transforms(cfg.data.image_size)
loaders = create_dataloaders(
    cfg.data.train_csv, cfg.data.val_csv, cfg.data.test_csv,
    data_root=cfg.data.data_root,
    train_transform=val_tf,
    val_transform=val_tf,
    batch_size=cfg.data.batch_size,
    num_workers=cfg.data.num_workers
)

# Load model with best checkpoint
model = create_model(cfg)
ckpt = torch.load("checkpoints/baseline_resnet50_best.pth", map_location="cuda")
model.load_state_dict(ckpt['model_state'])
model.cuda().eval()

print("Best checkpoint from epoch:", ckpt.get('epoch', 'unknown'))
print("Best val macro-F1:", ckpt.get('val_macro_f1', 'unknown'))
print("\nEvaluating on test set...")

# Run inference on test set
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels, _ in loaders['test']:
        images = images.cuda()
        outputs = model(images)
        preds = outputs.argmax(1).cpu()
        all_preds.append(preds)
        all_labels.append(labels)

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

# Print metrics
print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=['notdrowsy', 'drowsy']))
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))
print("\n[Row: True label | Column: Predicted label]")
print("[0=notdrowsy, 1=drowsy]")

## Step 11: Copy Checkpoints to Drive (for backup)

In [ ]:
# Create backup directory on Drive
!mkdir -p /content/drive/MyDrive/drowsiness-results/checkpoints

# Copy checkpoints
!cp checkpoints/baseline_resnet50_best.pth /content/drive/MyDrive/drowsiness-results/checkpoints/
!cp checkpoints/baseline_resnet50_last.pth /content/drive/MyDrive/drowsiness-results/checkpoints/

print("✅ Checkpoints backed up to Drive!")

## Step 12: Download Checkpoints (Optional)

If you want to download directly to your computer:

In [ ]:
from google.colab import files

# Create a zip of checkpoints
!zip -r baseline_resnet50_checkpoints.zip checkpoints/baseline_resnet50_*.pth

# Download the zip
files.download('baseline_resnet50_checkpoints.zip')

## Next Steps

After successful training:

1. **Download checkpoints** to your local machine
2. **Train EfficientNet-B0 baseline** for comparison:
   ```python
   !python src/training/train_baseline.py --config configs/baseline_efficientnet.yaml --device cuda
   ```
3. **Compare results** and create performance tables
4. **Begin ROI implementation** (next phase)

---

**Troubleshooting:**
- **CUDA not available:** Enable GPU runtime
- **Dataset not found:** Check Drive path in symlink command
- **Out of memory:** Reduce batch_size in config
- **Import errors:** Re-run pip install cell